In [ ]:
!pip3 install --user pandas boto3 mysql-connector-python scikit-learn numpy mord

In [94]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_absolute_error, cohen_kappa_score
import mord  # Importa a biblioteca mord para regressão ordinal
import time
from itertools import product
import sys
sys.path.append('../..')
from functions.db_inserts import *
from functions.aws_functions import *
from functions.select_cols_functions import *
from sklearn.preprocessing import StandardScaler

In [ ]:
env_mode = 'prd'

### Lendo arquivo fonte

In [96]:
bucket_name = "adoptai-trusted-prod-tcc"

In [97]:
if env_mode == 'dev':
    source = 'feature_engineering'
    df_trusted = pd.read_csv(f'../../../data/{source}.csv')
    df_trusted = apply_all_rules(df_trusted)
    s3 = [f'{source}.csv', df_trusted]
elif env_mode == 'prd':
    s3 = s3_csv_to_df(bucket_name)
    df_trusted = s3[1]

### Utilizando LabelEncoded nas colunas categóricas

In [98]:
df_trusted = adjust_categoric_values(df_trusted)

### Parametros de gravação

In [99]:
nome_fonte = s3[0]
nome_modelo = 'Regressão Logística Ordinal'
parameters = {
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1]
}

### Preparativos para executar o modelo

In [100]:

# Separar as features (X) e os rótulos (y)
x = df_trusted.iloc[:, :-1]
y = df_trusted.iloc[:, -1]

In [101]:
# Dividir os dados em conjuntos de treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [102]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [103]:
# Listas para armazenar métricas de desempenho e tempos de execução
accuracy = []
precision = []
recall = []
f1 = []
mae = []
qwk = []

start_times = []
end_times = []

In [104]:
for alpha in parameters['alpha']:
    
    start_time = start_times.append(time.time())
    
    model = mord.LogisticAT(alpha=alpha)  # Inicializa o modelo com alpha
    model.fit(x_train, y_train) 

    y_pred = model.predict(x_test)  # Faz previsões no conjunto de teste

    # Avaliar o modelo e armazenar as métricas
    accuracy.append(accuracy_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred, average=None))
    recall.append(recall_score(y_test, y_pred, average=None))
    f1.append(f1_score(y_test, y_pred, average=None))
    mae.append(mean_absolute_error(y_test, y_pred))
    qwk.append(cohen_kappa_score(y_test, y_pred, weights='quadratic'))  # Cálculo do QWK

    end_times.append(time.time())

c:\Users\erikp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\erikp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\erikp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [105]:
# Armazena as métricas calculadas
metrics = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1
}

In [106]:
# Prepara os dados para inserção no banco de dados
model = {'nome_modelo': nome_modelo, 'nome_fonte': nome_fonte}
execution = {
    'accuracy': accuracy, 
    'mae': mae,
    'qwk': qwk,
    'start_time': start_times, 
    'end_time': end_times
    }
hyperparams = parameters
desempenho = {
    'precision_values': metrics['precision'], 
    'recall_values': metrics['recall'], 
    'f1_values': metrics['f1_score']
    }

In [107]:
# Insere os valores calculados nas tabelas do banco de dados
insert_values(model, execution, hyperparams, desempenho, env=env_mode)

Executando em dev
Registros inseridos em dev
